# Gender classification using a Deformable ConvNet (Deformable Convolutional Network) combined with a Vision Transformer (ViT)

In [1]:
#import deeplake
import numpy as np

from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms, models

from transformers import ViTFeatureExtractor, ViTForImageClassification
import timm
#from mmdet.models.ops import DeformConv2d

from mmcv.ops import DeformConv2d

In [6]:
from dataprocessing import celebA_train_dataloader, celebA_val_dataloader, celebA_test_dataloader, adience_dataloader

|

Opening dataset in read-only mode as you don't have write permissions.


-

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/adience



-

...

/

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/celeb-a-test



\

hub://activeloop/celeb-a-test loaded successfully.



/Users/tulipmajumder/opt/anaconda3/lib/python3.9/site-packages/deeplake/integrations/pytorch/common.py:126: UserWarning: Decode method for tensors ['images'] is defaulting to numpy. Please consider specifying a decode_method in .pytorch() that maximizes the data preprocessing speed based on your transformation.
  warnings.warn(


In [12]:
class DeformableGenderClassifier(nn.Module):
    def __init__(self, num_classes):
        super(DeformableGenderClassifier, self).__init__()

        # Deformable Convolutional Layers
        self.deform_conv1 = DeformConv2d(3, 96, kernel_size=7, padding=0)
        self.deform_conv2 = DeformConv2d(96, 256, kernel_size=5, padding=0)
        self.deform_conv3 = DeformConv2d(256, 384, kernel_size=3, padding=0)

        # Vision Transformer (ViT)
        self.vit = timm.create_model('vit_base_patch16_224', pretrained=True)
        num_features = self.vit.head.in_features
        self.vit.head = nn.Identity()  # Remove the default classification head
        self.classifier = nn.Linear(num_features, num_classes)

    def forward(self, x):
        # Apply deformable convolution
        x = self.deform_conv1(x)
        x = self.deform_conv2(x)
        x = self.deform_conv3(x)

        # Pass the output through the Vision Transformer (ViT)
        x = self.vit(x)
        
        # Final classification
        x = self.classifier(x)
        return x

In [13]:
# Create an instance of the DeformableGenderClassifier with the desired number of classes
num_classes = 2  # For example, 2 classes: male and female
model = DeformableGenderClassifier(num_classes)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [14]:
model

DeformableGenderClassifier(
  (deform_conv1): DeformConv2d(in_channels=3,
  out_channels=96,
  kernel_size=(7, 7),
  stride=(1, 1),
  padding=(0, 0),
  dilation=(1, 1),
  groups=1,
  deform_groups=1,
  bias=False)
  (deform_conv2): DeformConv2d(in_channels=96,
  out_channels=256,
  kernel_size=(5, 5),
  stride=(1, 1),
  padding=(0, 0),
  dilation=(1, 1),
  groups=1,
  deform_groups=1,
  bias=False)
  (deform_conv3): DeformConv2d(in_channels=256,
  out_channels=384,
  kernel_size=(3, 3),
  stride=(1, 1),
  padding=(0, 0),
  dilation=(1, 1),
  groups=1,
  deform_groups=1,
  bias=False)
  (vit): VisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      (norm): Identity()
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (patch_drop): Identity()
    (norm_pre): Identity()
    (blocks): Sequential(
      (0): Block(
        (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
       

In [17]:
from sklearn.metrics import accuracy_score

num_epochs = 100

# Lists to store predictions and ground truth labels
gender_preds = []
age_preds = []
gender_gt = []
age_gt = []

# Lists to track losses
gender_losses = []
age_losses = []

for epoch in range(num_epochs):
    model.train()
    for inputs, age_labels, gender_labels in celebA_train_dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        
        # Calculate age and gender losses
        age_loss = criterion(outputs, age_labels)  
        gender_loss = criterion(outputs, gender_labels)  

        total_loss = age_loss + gender_loss
        total_loss.backward()
        optimizer.step()

        # Store losses for monitoring
        gender_losses.append(gender_loss.item())
        age_losses.append(age_loss.item())

    # Evaluate the model on the test set periodically (e.g., after each epoch)
    model.eval()
    with torch.no_grad():
        for inputs, age_labels, gender_labels in celebA_test_dataloader:
            outputs = model(inputs)
            age_preds.extend(outputs['age'].argmax(dim=1).cpu().numpy())
            gender_preds.extend(outputs['gender'].argmax(dim=1).cpu().numpy())
            age_gt.extend(age_labels.cpu().numpy())
            gender_gt.extend(gender_labels.cpu().numpy())

    # Calculate accuracy and print
    gender_accuracy = accuracy_score(gender_gt, gender_preds)
    age_accuracy = accuracy_score(age_gt, age_preds)
    print(f"Epoch [{epoch+1}/{num_epochs}] - Gender Accuracy: {gender_accuracy}, Age Accuracy: {age_accuracy}")


TypeError: forward() missing 1 required positional argument: 'offset'